# Analysis of School Data - Performance Indicators

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

#school_data_complete.head()

## District Summary

In [2]:
# total schools in dataset
total_schools = school_data_complete["school_name"].nunique()
total_schools

15

In [3]:
# total number of students
total_students = school_data_complete["student_name"].count()
total_students

39170

In [4]:
# school system budget (all schools)
total_budget = school_data_complete.loc[:, ["school_name","budget"]].drop_duplicates().sum()
total_budget=total_budget['budget']
total_budget

24649428

In [5]:
# average math score 
math_mean = school_data_complete['math_score'].mean()
math_mean

78.98537145774827

In [6]:
# average reading score
read_mean = school_data_complete['reading_score'].mean()
read_mean

81.87784018381414

In [7]:
# non arithmetic mean - might be okay in this example since all students should have scores for both tests
pass_mean = (math_mean + read_mean)/2
pass_mean

80.43160582078121

In [8]:
# math pass rate > 70
pass_math = len(school_data_complete[school_data_complete['math_score']>=70])/total_students*100
pass_math

74.9808526933878

In [9]:
# reading pass rate > 70
pass_read = len(school_data_complete[school_data_complete['reading_score']>=70])/total_students*100
pass_read

85.80546336482001

In [10]:
# create dataframe from above scalars
data={
    'Total Schools':total_schools,
    'Total Students':total_students,
    'Total Budget':total_budget,
    'Average Math Score':math_mean,
    'Average Reading Score':read_mean,
    'Overall % Pass':pass_mean,
    '% Math Pass':pass_math,
    '% Read Pass':pass_read
}

# need to put dictionary in list; defining the index is unnecessary
summary_df=pd.DataFrame([data], index=[0])
summary_df.head()

Total Schools  Total Students  Total Budget  Average Math Score  \
0             15           39170      24649428           78.985371   

   Average Reading Score  Overall % Pass  % Math Pass  % Read Pass  
0               81.87784       80.431606    74.980853    85.805463

In [11]:
# formatting; nice use of dictionary
# using `style.format` is probably better than using `.map` since map converts everything to strings. 
summary_df.style.format({'Total Students': "{0:,.0f}",'Total Budget': "${0:,.2f}"})

## School Summary

In [12]:
# make dataframe with school as the index
main_df = school_data_complete.loc[:, ["school_name","type"]].sort_values(by=['school_name']).drop_duplicates()
main_df.set_index(['school_name'], inplace=True)
main_df.rename(columns = {"type":"School Type"}, inplace=True)
#main_df.head()

In [13]:
# series of total students
count_values = school_data_complete.loc[:, ["school_name","student_name"]].groupby("school_name").count()
count_values.rename(columns = {"student_name":"Total Students"}, inplace=True)
main_df=main_df.merge(count_values, on=['school_name'])

In [14]:
# series of school budget
total_budget = school_data_complete.loc[:, ["school_name","budget"]].drop_duplicates()
total_budget.rename(columns = {"budget":"Total School Budget"}, inplace=True)
total_budget.set_index("school_name", inplace=True)
main_df=main_df.merge(total_budget, on=['school_name'])
#main_df.head()

In [15]:
# calculated the school budget per student 
main_df['Per Student Budget'] = main_df['Total School Budget']/main_df['Total Students']
#main_df.head()

In [16]:
# create and merge dataframe of test scores for each school
mean_values = school_data_complete.loc[:, ["school_name","student_name","math_score", "reading_score"]].groupby("school_name").mean()
mean_values.rename(columns = {"math_score":"Average Math Score", "reading_score":"Average Reading Score"}, inplace=True)
main_df=main_df.merge(mean_values, on=['school_name'])
#main_df.head()

In [17]:
# make dataframe of math scores > 70 per school then merge
pass_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby(['school_name']).count()
pass_percent_math = pass_math['Student ID']/main_df['Total Students']*100
pass_percent_math = pass_percent_math.to_frame("% Passing Math")
main_df=main_df.merge(pass_percent_math, on=['school_name'])
#main_df.head()

In [18]:
# make dataframe of reading scores > 70 per school then merge
pass_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby(['school_name']).count()
pass_percent_read = pass_read['Student ID']/main_df['Total Students']*100
pass_percent_read = pass_percent_read.to_frame("% Passing Reading")
main_df=main_df.merge(pass_percent_read, on=['school_name'])
#main_df.head()

In [19]:
# calculated the better way of doing the estimate  
pass_read_and_math = school_data_complete[ (school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('school_name').count()

percent_pass_read_and_math = pass_read_and_math['student_name']/main_df['Total Students']*100


In [20]:
# calculated non-weighted percent overall passing 
# this is not actually the correct way since it is not weighted.
# this value will always be between the two test scores (an average)
main_df['Overall % Passing Rate']= (main_df['% Passing Math'] + main_df['% Passing Reading'])/2

# display the better way of doing the estimate  
main_df['Better Overall # Passsing'] = percent_pass_read_and_math
main_df.head()

School Type  Total Students  Total School Budget  \
school_name                                                             
Bailey High School      District            4976              3124928   
Cabrera High School      Charter            1858              1081356   
Figueroa High School    District            2949              1884411   
Ford High School        District            2739              1763916   
Griffin High School      Charter            1468               917500   

                      Per Student Budget  Average Math Score  \
school_name                                                    
Bailey High School                 628.0           77.048432   
Cabrera High School                582.0           83.061895   
Figueroa High School               639.0           76.711767   
Ford High School                   644.0           77.102592   
Griffin High School                625.0           83.351499   

                      Average Reading Score  % Passing Math  \
school_name                                                   
Bailey High School                81.033963       66.680064   
Cabrera High School               83.975780       94.133477   
Figueroa High School              81.158020       65.988471   
Ford High School                  80.746258       68.309602   
Griffin High School               83.816757       93.392371   

                      % Passing Reading  Overall % Passing Rate  \
school_name                                                       
Bailey High School            81.933280               74.306672   
Cabrera High School           97.039828               95.586652   
Figueroa High School          80.739234               73.363852   
Ford High School              79.299014               73.804308   
Griffin High School           97.138965               95.265668   

                      Better Overall # Passsing  
school_name                                      
Bailey High School                    54.642283  
Cabrera High School                   91.334769  
Figueroa High School                  53.204476  
Ford High School                      54.289887  
Griffin High School                   90.599455

## Top Performing Schools (By Passing Rate)

In [21]:
# Sort and display the top five schools in overall passing rate
main_df.sort_values('Overall % Passing Rate', ascending=False).head()

School Type  Total Students  Total School Budget  \
school_name                                                            
Cabrera High School     Charter            1858              1081356   
Thomas High School      Charter            1635              1043130   
Pena High School        Charter             962               585858   
Griffin High School     Charter            1468               917500   
Wilson High School      Charter            2283              1319574   

                     Per Student Budget  Average Math Score  \
school_name                                                   
Cabrera High School               582.0           83.061895   
Thomas High School                638.0           83.418349   
Pena High School                  609.0           83.839917   
Griffin High School               625.0           83.351499   
Wilson High School                578.0           83.274201   

                     Average Reading Score  % Passing Math  % Passing Reading  \
school_name                                                                     
Cabrera High School              83.975780       94.133477          97.039828   
Thomas High School               83.848930       93.272171          97.308869   
Pena High School                 84.044699       94.594595          95.945946   
Griffin High School              83.816757       93.392371          97.138965   
Wilson High School               83.989488       93.867718          96.539641   

                     Overall % Passing Rate  Better Overall # Passsing  
school_name                                                             
Cabrera High School               95.586652                  91.334769  
Thomas High School                95.290520                  90.948012  
Pena High School                  95.270270                  90.540541  
Griffin High School               95.265668                  90.599455  
Wilson High School                95.203679                  90.582567

## Bottom Performing Schools (By Passing Rate)

In [22]:
# Sort and display the five worst-performing schools
main_df.sort_values('Overall % Passing Rate').head()

School Type  Total Students  Total School Budget  \
school_name                                                              
Rodriguez High School    District            3999              2547363   
Figueroa High School     District            2949              1884411   
Huang High School        District            2917              1910635   
Johnson High School      District            4761              3094650   
Ford High School         District            2739              1763916   

                       Per Student Budget  Average Math Score  \
school_name                                                     
Rodriguez High School               637.0           76.842711   
Figueroa High School                639.0           76.711767   
Huang High School                   655.0           76.629414   
Johnson High School                 650.0           77.072464   
Ford High School                    644.0           77.102592   

                       Average Reading Score  % Passing Math  \
school_name                                                    
Rodriguez High School              80.744686       66.366592   
Figueroa High School               81.158020       65.988471   
Huang High School                  81.182722       65.683922   
Johnson High School                80.966394       66.057551   
Ford High School                   80.746258       68.309602   

                       % Passing Reading  Overall % Passing Rate  \
school_name                                                        
Rodriguez High School          80.220055               73.293323   
Figueroa High School           80.739234               73.363852   
Huang High School              81.316421               73.500171   
Johnson High School            81.222432               73.639992   
Ford High School               79.299014               73.804308   

                       Better Overall # Passsing  
school_name                                       
Rodriguez High School                  52.988247  
Figueroa High School                   53.204476  
Huang High School                      53.513884  
Johnson High School                    53.539172  
Ford High School                       54.289887

## Math Scores by Grade

In [24]:
#Create dataframes for each set of data
g9_math = school_data_complete[school_data_complete['grade'] == '9th'].groupby(['school_name'])
g9_math_vals = g9_math[['school_name','math_score']].mean()
g10_math = school_data_complete[school_data_complete['grade'] == '10th'].groupby(['school_name'])
g10_math_vals = g10_math[['school_name','math_score']].mean()
g11_math = school_data_complete[school_data_complete['grade'] == '11th'].groupby(['school_name'])
g11_math_vals = g11_math[['school_name','math_score']].mean()
g12_math = school_data_complete[school_data_complete['grade'] == '12th'].groupby(['school_name'])
g12_math_vals = g12_math[['school_name','math_score']].mean()

#Merge the dataframes into one dataframe and rename after each
df_1=g9_math_vals.merge(g10_math_vals, on='school_name')
df_1.rename(columns={'math_score_x':"9th", 'math_score_y':"10th"}, inplace=True)
df_1=df_1.merge(g11_math_vals, on='school_name')
df_1.rename(columns={'math_score':"11th"}, inplace=True)
df_1=df_1.merge(g12_math_vals, on='school_name')
df_1.rename(columns={'math_score':"12th"}, inplace=True)

df_1.head()

9th       10th       11th       12th
school_name                                                     
Bailey High School    77.083676  76.996772  77.515588  76.492218
Cabrera High School   83.094697  83.154506  82.765560  83.277487
Figueroa High School  76.403037  76.539974  76.884344  77.151369
Ford High School      77.361345  77.672316  76.918058  76.179963
Griffin High School   82.044010  84.229064  83.842105  83.356164

## Reading Score by Grade 

In [26]:
#Create dataframes for each set of data
g9_read = school_data_complete[school_data_complete['grade'] == '9th'].groupby(['school_name'])
g9_read_vals = g9_read[['school_name','reading_score']].mean()
g10_read = school_data_complete[school_data_complete['grade'] == '10th'].groupby(['school_name'])
g10_read_vals = g10_read[['school_name','reading_score']].mean()
g11_read = school_data_complete[school_data_complete['grade'] == '11th'].groupby(['school_name'])
g11_read_vals = g11_read[['school_name','reading_score']].mean()
g12_read = school_data_complete[school_data_complete['grade'] == '12th'].groupby(['school_name'])
g12_read_vals = g12_read[['school_name','reading_score']].mean()

#Merge the dataframes into one dataframe and rename after each
df_1=g9_read_vals.merge(g10_read_vals, on='school_name')
df_1.rename(columns={'reading_score_x':"9th", 'reading_score_y':"10th"}, inplace=True)
df_1=df_1.merge(g11_read_vals, on='school_name')
df_1.rename(columns={'reading_score':"11th"}, inplace=True)
df_1=df_1.merge(g12_read_vals, on='school_name')
df_1.rename(columns={'reading_score':"12th"}, inplace=True)


df_1.head()

9th       10th       11th       12th
school_name                                                     
Bailey High School    81.303155  80.907183  80.945643  80.912451
Cabrera High School   83.676136  84.253219  83.788382  84.287958
Figueroa High School  81.198598  81.408912  80.640339  81.384863
Ford High School      80.632653  81.262712  80.403642  80.662338
Griffin High School   83.369193  83.706897  84.288089  84.013699

## Scores by School Spending

In [27]:
# Sample bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [28]:
#This is simple average of category values and is not weighted
filter_df = main_df.loc[:, ['Per Student Budget','Average Math Score', 'Average Reading Score','% Passing Math', '% Passing Reading', 'Overall % Passing Rate']]
filter_df["Spending Ranges (Per Student)"] = pd.cut(filter_df['Per Student Budget'], spending_bins, labels=group_names)
filter_df.drop(columns=['Per Student Budget'], inplace=True)
filter_df = filter_df.groupby(['Spending Ranges (Per Student)'])
filter_df.mean()

Average Math Score  Average Reading Score  \
Spending Ranges (Per Student)                                              
<$585                                   83.455399              83.933814   
$585-615                                83.599686              83.885211   
$615-645                                79.079225              81.891436   
$645-675                                76.997210              81.027843   

                               % Passing Math  % Passing Reading  \
Spending Ranges (Per Student)                                      
<$585                               93.460096          96.610877   
$585-615                            94.230858          95.900287   
$615-645                            75.668212          86.106569   
$645-675                            66.164813          81.133951   

                               Overall % Passing Rate  
Spending Ranges (Per Student)                          
<$585                                       95.035486  
$585-615                                    95.065572  
$615-645                                    80.887391  
$645-675                                    73.649382

## Scores by School Size

In [29]:
# Sample bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [30]:
#Copy and paste and replace of above code
filter2_df = main_df.loc[:, ['Total Students','Average Math Score', 'Average Reading Score','% Passing Math', '% Passing Reading', 'Overall % Passing Rate']]
filter2_df["School Size"] = pd.cut(filter2_df['Total Students'], size_bins, labels=group_names)
filter2_df.drop(columns=['Total Students'], inplace=True)
filter2_df = filter2_df.groupby(['School Size'])
filter2_df.mean()

Average Math Score  Average Reading Score  % Passing Math  \
School Size                                                                     
Small (<1000)                83.821598              83.929843       93.550225   
Medium (1000-2000)           83.374684              83.864438       93.599695   
Large (2000-5000)            77.746417              81.344493       69.963361   

                    % Passing Reading  Overall % Passing Rate  
School Size                                                    
Small (<1000)               96.099437               94.824831  
Medium (1000-2000)          96.790680               95.195187  
Large (2000-5000)           82.766634               76.364998

## Scores by School Type

In [31]:
# Perform the same operations as above, based on school type.
#This is a variation of above but the binning is not really necessary...
filter3_df = main_df.loc[:, ['School Type','Average Math Score', 'Average Reading Score','% Passing Math', '% Passing Reading', 'Overall % Passing Rate']]
filter3_df=filter3_df.groupby(['School Type'])
filter3_df.mean()

Average Math Score  Average Reading Score  % Passing Math  \
School Type                                                              
Charter               83.473852              83.896421       93.620830   
District              76.956733              80.966636       66.548453   

             % Passing Reading  Overall % Passing Rate  
School Type                                             
Charter              96.586489               95.103660  
District             80.799062               73.673757